To get started with this notebook, you should first configure your Python environment using your preferred environment management system (e.g., Conda, virtualenv, pipenv). If you don’t already use one, we provide a helper script that:

1. Detects your operating system  
2. Installs Miniconda automatically on macOS or Linux(Windows users will need to install Miniconda manually following the instructions at https://docs.conda.io/en/latest/miniconda.html)

First, make the installer executable and run it in your terminal:

```bash
chmod +x install-miniconda.sh
./install-miniconda.sh
```

Once Miniconda is installed (or if you already have Conda), create the environment from the provided `environment.yml`:

```bash
conda env create -f environment.yml
```

After the environment is created, activate it and launch Jupyter Notebook:

```bash
conda activate ot_env
jupyter notebook
```

Now you’re ready to run the cells in this notebook!


In [1]:
import requests
from ipyleaflet import Map, DrawControl, basemaps, TileLayer, GeoJSON, LegendControl
import rasterio
import matplotlib.pyplot as plt
import rioxarray as rio
import numpy as np
from osgeo import gdal, osr
from matplotlib.colors import LinearSegmentedColormap
import pyproj
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import sys

# Table of Contents

- [1. Define area of interest](#1-define-area-of-interest)  
    - [Option 1: Define bounds manually or using an uploaded shapefile](#option-1-define-bounds-manually-or-using-an-uploaded-shapefile)  
    - [Option 2: Draw a bounding box on an interactive map](#option-2-draw-a-bounding-box-on-an-interactive-map)  
- [2. Use OT catalog to find datasets](#2-use-ot-catalog-to-find-datasets)  
- [3. Display results](#3-display-results)  


## 1. Define area of interest 

### Option 1: Define bounds manually or using an uploaded shapefile

### Option 2: Draw a bounding box on an interactive map

In [3]:
# Initialize global variables to store bounding box coordinates
global_bounds = {'south': 0, 'north': 0, 'west': 0, 'east': 0, 'polygon': 0,}

def handle_draw(self, action, geo_json):
    # Extract the bounding box coordinates from the drawn rectangle
    coords = geo_json['geometry']['coordinates'][0]
    global_bounds['south'] = coords[0][1]
    global_bounds['west'] = coords[0][0]
    global_bounds['north'] = coords[2][1]
    global_bounds['east'] = coords[2][0]
    global_bounds['polygon'] = geojson_to_wkt(geo_json['geometry'])
    
    print(f"Bounds updated: {global_bounds}")

# Initialize the map
m = Map(center=(39.8283, -98.5795), zoom=3, basemap=basemaps.Esri.WorldTopoMap)

# Set up the drawing tool
draw_control = DrawControl(rectangle={'shapeOptions': {'color': '#fca45d'}})
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.circle = {}
draw_control.circlemarker = {}
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

# Fetch the GeoJSON data from the URL
three_dep_url = "https://raw.githubusercontent.com/OpenTopography/Data_Catalog_Spatial_Boundaries/main/usgs_3dep_boundaries.geojson"
response1 = requests.get(three_dep_url)
geojson_three_dep_data = response1.json()

noaa_url = "https://raw.githubusercontent.com/OpenTopography/Data_Catalog_Spatial_Boundaries/main/noaa_coastal_lidar_boundaries.geojson"
response2 = requests.get(noaa_url)
geojson_noaa_data = response2.json()

open_topo_url = "https://raw.githubusercontent.com/OpenTopography/Data_Catalog_Spatial_Boundaries/main/OT_PC_boundaries.geojson"
response3 = requests.get(open_topo_url)
geojson_ot_data = response3.json()

# Create a GeoJSON layer and add it to the map
geojson_three_dep_layer = GeoJSON(data=geojson_three_dep_data, name="3DEP datasets", style={'color': '#228B22'})
m.add_layer(geojson_three_dep_layer)

geojson_noaa_layer = GeoJSON(data=geojson_noaa_data, name="NOAA datasets", style={'color': '#0000CD'})
m.add_layer(geojson_noaa_layer)

geojson_ot_layer = GeoJSON(data=geojson_ot_data, name="OpenTopography datasets", style={'color': '#fca45d'})
m.add_layer(geojson_ot_layer)

# Create a legend and add it to the map
legend = LegendControl({
    "3DEP datasets":"#228B22",
    "NOAA datasets":"#0000CD",
    "OpenTopography datasets": "#fca45d"
    }, name="Legend", position="topright")
m.add_control(legend)

display(m)

Map(center=[39.8283, -98.5795], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [ ]:
def geojson_to_wkt(geojson):
    # Ensure the input is a Polygon
    if geojson['type'] != 'Polygon':
        raise ValueError("Input must be a Polygon")

    # Extract coordinates
    coordinates = geojson['coordinates']
    
    # Convert coordinates to WKT string
    wkt_coordinates = ', '.join(
        f"{', '.join(f'{x}, {y}' for x, y in polygon)}"
        for polygon in coordinates
    )
    
    return wkt_coordinates
    
# Initialize global variables to store bounding box coordinates
global_bounds = {'south': 0, 'north': 0, 'west': 0, 'east': 0, 'polygon': 0,}

def handle_draw(self, action, geo_json):
    # Extract the bounding box coordinates from the drawn rectangle
    coords = geo_json['geometry']['coordinates'][0]
    global_bounds['south'] = coords[0][1]
    global_bounds['west'] = coords[0][0]
    global_bounds['north'] = coords[2][1]
    global_bounds['east'] = coords[2][0]
    global_bounds['polygon'] = geojson_to_wkt(geo_json['geometry'])
    
    print(f"Bounds updated: {global_bounds}")

# Initialize the map
m = Map(center=(39.8283, -98.5795), zoom=3, basemap=basemaps.Esri.WorldTopoMap)

# Set up the drawing tool
draw_control = DrawControl(rectangle={'shapeOptions': {'color': '#fca45d'}})
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.circle = {}
draw_control.circlemarker = {}
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

m

Map(center=[39.8283, -98.5795], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

Bounds updated: {'south': 39.359785, 'north': 43.703622, 'west': -115.817871, 'east': -106.946411, 'polygon': '-115.817871, 39.359785, -115.817871, 43.703622, -106.946411, 43.703622, -106.946411, 39.359785, -115.817871, 39.359785'}


## 2. Use OT catalog to find datasets

In [ ]:
# Ensure base_url matches your API's base URL
base_url = "https://portal.opentopography.org//API"
endpoint = "/otCatalog"
output_format = "json"

# Prepare the query parameters using the bounds stored in global_bounds
params = {
    "productFormat": "PointCloud",  # Data product Format (optional) - Point Cloud or Raster. Defaults to all data products
    "minx": global_bounds['west'], # WGS 84 bounding box west coordinates. Required if polygon parameter is not provided.
    "miny": global_bounds['south'], # WGS 84 bounding box south coordinates. Required if polygon parameter is not provided.
    "maxx": global_bounds['east'], # WGS 84 bounding box east coordinates. Required if polygon parameter is not provided.
    "maxy": global_bounds['north'], # WGS 84 bounding box north coordinates. Required if polygon parameter is not provided.
    "polygon": global_bounds['polygon'], # A string of points to make up a polygon (WKT format). The first and last points should match to complete the polygonal shape. If this parameter is provided, the bounding box parameters minx, miny, maxx, and maxy will be ignored.
    "detail": False, # Show detailed metadata (optional). Defaults is false and shows specific metadata information.
    "outputFormat": f"{output_format}", # Output Format (optional) - Defaults to json if parameter is not provided. Other available formats - xml
    "include_federated": True, #Include federated datasets available via OpenTopography (e.g. USGS 3DEP Catalog)
    
}

# Make the GET request to the API
response = requests.get(url=base_url + endpoint, params=params)

# Check if the request was successful
if response.status_code == 200:
    filename = f'results.{output_format}' 
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"File has been saved as '{filename}'.")
else:
    print(f"Error: {response.status_code}")

File has been saved as 'results.json'.


In [ ]:
global_bounds

{'south': 39.359785,
 'north': 43.703622,
 'west': -115.817871,
 'east': -106.946411,
 'polygon': '-115.817871, 39.359785, -115.817871, 43.703622, -106.946411, 43.703622, -106.946411, 39.359785, -115.817871, 39.359785'}

## 3. Display results

In [ ]:
m

Map(bottom=23874.0, center=[44.61393394730628, -90.73883056640626], controls=(ZoomControl(options=['position',…

In [ ]:
"/otCatalog" : {
      "get" : {
        "tags" : [ "Public" ],
        "summary" : "Bounding box-based catalog search of topography datasets hosted by OpenTopography.",
        "operationId" : "getOtCatalog",
        "parameters" : [
        {
          "name" : "productFormat",
          "in" : "query",
          "description" : "Data product Format (optional) - Point Cloud or Raster. Defaults to all data products",
          "required" : false,
          "schema" : {
            "type" : "string",
            "enum": ["PointCloud", "Raster"]
          },
          "example" : "PointCloud"
        }, {
          "name" : "minx",
          "in" : "query",
          "description" : "WGS 84 bounding box west coordinates. <sup>*</sup>Required if polygon parameter is not provided.",
          "required" : false,
          "schema" : {
            "maximum" : 180,
            "minimum" : -180,
            "type" : "number"
          },
          "example" : -123.2
        }, {
          "name" : "miny",
          "in" : "query",
          "description" : "WGS 84 bounding box south coordinates. <sup>*</sup>Required if polygon parameter is not provided.",
          "required" : false,
          "schema" : {
            "maximum" : 90,
            "minimum" : -90,
            "type" : "number"
          },
          "example" : 44.8
        }, {
          "name" : "maxx",
          "in" : "query",
          "description" : "WGS 84 bounding box east coordinates. <sup>*</sup>Required if polygon parameter is not provided.",
          "required" : false,
          "schema" : {
            "maximum" : 180,
            "minimum" : -180,
            "type" : "number"
          },
          "example" : -121.9
        }, {
          "name" : "maxy",
          "in" : "query",
          "description" : "WGS 84 bounding box north coordinates. <sup>*</sup>Required if polygon parameter is not provided.",
          "required" : false,
          "schema" : {
            "maximum" : 90,
            "minimum" : -90,
            "type" : "number"
          },
          "example" : 45.7
        }, {
          "name" : "polygon",
          "in" : "query",
          "description" : "A string of points to make up a polygon (WKT format). The first and last points should match to complete the polygonal shape. <sup>*</sup>If this parameter is provided, the bounding box parameters: minx, miny, maxx, and maxy will be ignored. <br><br><i>Example: -117.5,32.5,-117.5,33.1,-116.7,33.1,-116.7,32.5,-117.0,32.3,-117.5,32.5</i>",
          "schema" : {
            "type" : "string"
          }
        }, {
          "name" : "detail",
          "in" : "query",
          "description" : "Show detailed metadata (optional). Defaults is false and shows specific metadata information.",
          "schema" : {
            "type" : "boolean"
          },
          "example" : false
        }, {
          "name" : "outputFormat",
          "in" : "query",
          "description" : "Output Format (optional) - Defaults to json if parameter is not provided. Other available formats - xml",
          "schema" : {
            "type" : "string",
            "enum" : [ "json", "xml"],
            "default" : "json"
          },
          "example" : "xml"
        }, {
          "name" : "include_federated",
          "in" : "query",
          "description" : "Include federated datasets available via OpenTopography (e.g. USGS 3DEP Catalog)",
          "schema" : {
            "type" : "boolean"
          },
          "example" : false
        }],
        "responses" : {
          "200" : {
            "description" : "OK"
          },
          "204" : {
            "description" : "No Data"
          },
          "400" : {
            "description" : "Bad request"
          },
          "500" : {
            "description" : "Internal error"
          }
        }
      }
    }
  }
}

In [ ]:
from fastkml import kml

def print_kml_properties(kml_file_path):
    with open(kml_file_path, 'r', encoding='utf-8') as kml_file:
        kml_content = kml_file.read()

    k = kml.KML()
    k.from_string(kml_content.encode('utf-8'))

    def _print_properties(feature, level=0):
        indent = ' ' * level * 4  # Indentation for readability
        if hasattr(feature, 'features'):  # Document or Folder
            print(f"{indent}{feature.__class__.__name__}: {feature.name}")
            for sub_feature in feature.features():
                _print_properties(sub_feature, level + 1)
        else:  # Placemark or other feature type
            print(f"{indent}{feature.__class__.__name__}: {feature.name}")
            # Print properties like description, styleUrl, etc.
            print(f"{indent}  Description: {feature.description}")
            # If the feature has extended data or custom properties, print them
            if hasattr(feature, 'extended_data') and feature.extended_data:
                for data in feature.extended_data.elements:
                    print(f"{indent}  Extended Data: {data.name} = {data.value}")

    for feature in k.features():
        _print_properties(feature)

# Example usage
kml_file_path = '/Users/cassandrabrigham/Documents/POSTDOC/Code/OT code/OT_API_notebooks/GeoJSON conversion/dataset_extent_all.kml'
print_kml_properties(kml_file_path)

Document: OpenTopo KML
    Placemark: None
      Description: 
            
            <div style="width: 400px;">
                <h3>OpenTopography</h3>
                <table>
                    <tr><td><strong>Dataset&nbsp;Name:</strong></td><td style="padding-left:5px; "><a href="https://portal.opentopography.org/datasetMetadata?otCollectionID=OT.062005.2871.1">Northern San Andreas Fault, CA</a></td></tr>
                    <tr><td><strong>Platform:</strong></td><td style="padding-left:5px;">Airborne Lidar</td></tr>
                    <tr><td><strong>Survey&nbsp;Date:</strong></td><td style="padding-left:5px; ">02/01/2003</td></tr>
                </table>
            </div>
            
            
    Placemark: None
      Description: 
            
            <div style="width: 400px;">
                <h3>OpenTopography</h3>
                <table>
                    <tr><td><strong>Dataset&nbsp;Name:</strong></td><td style="padding-left:5px; "><a href="https://portal.o

In [ ]:
from fastkml import kml

def list_kml_dataset_names(kml_file_path):
    with open(kml_file_path, 'r', encoding='utf-8') as kml_file:
        kml_content = kml_file.read()

    k = kml.KML()
    k.from_string(kml_content.encode('utf-8'))

    dataset_names = []

    def _extract_properties(feature):
        if hasattr(feature, 'features'):  # Document or Folder
            for sub_feature in feature.features():
                _extract_properties(sub_feature)
        else:  # Placemark
            # Assuming dataset name might be stored in the name or extended data
            if feature.name:
                dataset_names.append(feature.name)
            if hasattr(feature, 'extended_data') and feature.extended_data:
                for data in feature.extended_data.elements:
                    if data.name == 'dataset name':  # or adjust the condition based on the actual data
                        dataset_names.append(data.value)

    for feature in k.features():
        _extract_properties(feature)

    return dataset_names

# Replace 'path/to/your/file.kml' with the actual path to your KML file
kml_file_path = '/Users/cassandrabrigham/Documents/POSTDOC/Code/OT code/OT_API_notebooks/GeoJSON conversion/dataset_extent_all.kml'
dataset_names = list_kml_dataset_names(kml_file_path)
print(dataset_names)

[]


In [ ]:
import xmltodict

kml_file_path = '/Users/cassandrabrigham/Documents/POSTDOC/Code/OT code/OT_API_notebooks/GeoJSON conversion/dataset_extent_OT.112011.26912.3.kml'

with open(kml_file_path, 'r', encoding='utf-8') as file:
        kml_content = file.read()
        
kml_dict = xmltodict.parse(kml_content)